In [1]:
push!(LOAD_PATH, pwd())
using FCV
using Plots, LinearAlgebra, Random
rng = MersenneTwister(7);

# Initialization

In [ ]:
N = 2^10
M = N
s = 3
What = [ abs(freq+1).^s for freq in -N/2:N/2-1 ]
fun = x -> peaks(x, 0)
nodes = (-N/2:N/2-1)/N
function peaks(x, y) # peaks sample function from matlab
  x *= 6
  y *= 6
  z = 3*(1-x)^2*exp(-(x^2) - (y+1)^2) -
    10*(x/5 - x^3 - y^5)*exp(-x^2-y^2) -
    1/3*exp(-(x+1)^2 - y^2) 
  return z
end
f = fun.(nodes)
sigma = 0.1
fe = f+sigma*(maximum(f)-minimum(f))*randn(rng, size(f))

lambda = [ exp(t) for t in range(-20,0, length = 50) ]
cv = similar(lambda)
l2err = similar(lambda)

@time fcv = fcv_t_equispaced(1, fe);

# Computations

In [ ]:
@time for i in eachindex(lambda)
  res = compute(fcv, lambda[i]*What)
  cv[i] = res.gcv
  l2err[i] = 1/M*norm(f-res.f_r)
end

res = compute(fcv, lambda[argmin(cv)]*What);

# Plotting

In [ ]:
plot(lambda, cv,
  color = :black,
  axis = (:log10),
  xlabel = "lambda",
  ylabel = "cv",
  legend = :none)
plot!(twinx(), lambda, l2err,
  color = :orange,
  legend = :none,
  axis = (:log10),
  ylabel = "l2err",
  guidefontcolor = :orange)

In [ ]:
 scatter(nodes, fe,
    markersize = 1,
    label = "noisy data")
  plot!(nodes, real(res.f_r),
    label = "reconstruction")